In [1]:
from utilities import *
from pyspeckit.spectrum.models import lte_molecule
from astroquery.linelists.cdms import CDMS
import numpy as np
import astropy.units as u
from astroquery.splatalogue import Splatalogue
import pickle

block_reduce was moved to the astropy.nddata.blocks module.  Please update your import statement.


In [15]:
freq_min=216.47*u.GHz
freq_max=218.8*u.GHz
def compute_cdms_aij(lgint,nu,g,elower,eupper,q,T=300*u.K):
    actualint=10**lgint*u.nm**2*u.MHz
    return (actualint/(4.16231e-5*nu*g))*((64*(np.pi**4)*(nu**3)*q)/(3*h*(c**3)))*(np.exp(-elower/(k*T))-np.exp(-eupper/(k*T)))**-1

def compute_mu2S(intensity,nu,g,elower,eupper,q,T=300*u.K):
    return ((2.40251e4*intensity*q)/g)*(nu**-1)*(np.exp(-elower/(k*T))-np.exp(-eupper/(k*T)))**-1

def pickett_aul(intensity,nu,g,elower,eupper,q,T=300*u.K):
    return (intensity*(nu**2)*(q/g)*((np.exp(-elower/(k*T))-np.exp(-eupper/(k*T)))**-1)*2.7964e-16).value*u.Hz

def absorption_crossection(intensity,nu):#Equation 8 from Pickett et al. 1998 https://www.sciencedirect.com/science/article/pii/S0022407398000910
    return (intensity/nu)*2.99792e-9

def pickett_aul_approx(intensity,nu,g,eupper,q,T=300*u.K):
    return intensity*nu*(q/g)*np.exp(-eupper/(k*T))*1.748e-9

In [3]:
maintable=CDMS.query_lines(min_frequency=freq_min,max_frequency=freq_max,min_strength=-500,molecule='046524 C2H5OH,v=0',get_query_payload=False)

In [4]:
testline=maintable[6]
testlgint=testline['LGINT']
testint=10**testlgint*u.nm**2*u.MHz

testnu=testline['FREQ']*u.MHz
testg=testline['GUP']

testelo_cm=testline['ELO']*u.cm**-1
testelo_J=(h*c*testelo_cm).to('J')
testdeltae=(h*testnu).to('J')
testeup_J=testelo_J+testdeltae

In [5]:
catdir=CDMS.get_species_table()
c2h5oh=catdir[catdir['TAG'] == 46524]
qrot300=10**c2h5oh['lg(Q(300))']
qrot300

45603.691595129596


In [6]:
sigma_abs=absorption_crossection(testint,testnu)
sigma_abs.to('cm2')

<Quantity 3.55319162e-33 cm2>

In [11]:
ints=10**maintable['LGINT']
nus=maintable['FREQ']
degs=maintable['GUP']
elo_cm=maintable['ELO']
elo_J=(h*c*elo_cm).to('J')
deltae=(h*nus).to('J')
eup_J=elo_J+deltae

In [26]:
qrot300

45603.691595129596


In [16]:
pickett_aul(ints,nus.to('MHz'),degs,elo_J,eup_J,qrot300,300*u.K)

<Quantity [2.55395124e-06, 5.61469858e-07, 3.15957347e-06, 3.15957347e-06,
           9.25107834e-06, 9.25107834e-06, 3.64958318e-05, 1.80210301e-07,
           1.64436456e-06, 1.64436456e-06, 1.13409939e-06, 1.13409939e-06,
           6.07654572e-07, 2.03350151e-06, 2.03350152e-06, 9.27545622e-06,
           9.27545625e-06, 6.00732461e-06, 3.65523016e-05, 1.03801633e-07,
           1.23601369e-07, 2.08258133e-06, 2.08258133e-06, 6.11826455e-06,
           6.11826455e-06, 6.12507147e-07, 2.08412980e-08, 3.83675567e-06,
           1.03268399e-07, 2.27652471e-05, 3.17763417e-07, 1.38779574e-07,
           1.51236956e-05, 4.29205472e-08, 1.51260133e-05, 6.94950524e-07,
           6.95144949e-07, 1.35914765e-07, 5.36274519e-07, 9.21743553e-05,
           1.13905134e-05, 1.14558317e-05, 1.14594870e-05, 1.45157674e-07,
           1.46468534e-07, 2.16439595e-06, 2.16441400e-06, 1.51866336e-05,
           7.65588716e-06, 6.27214028e-05, 5.12016464e-08, 5.31123215e-07,
           1.50457179e-05

In [17]:
pickett_aul_approx(testint,testnu.to('MHz'),testg,testeup_J,qrot300,300*u.K)

<Quantity [1.82813256e-05] MHz2 nm2>

In [19]:
a=pickett_aul(testint,testnu.to('MHz'),testg,testelo_J,testeup_J,qrot300,300*u.K)
a.decompose()

<Quantity [3.64958318e-05] 1 / s>

In [37]:
picklepath='/blue/adamginsburg/d.jeff/SgrB2DSreorg/field10/C2H5OH/DSi/chisquare_goodnessoffit_2_fwhmslabs/C2H5OHlinesdict.obj'
a=open(picklepath,'rb')
pick=pickle.load(a)
pick['spw0']
#testline

{'8_4&4-7_3&4&g+': {'freq': <Quantity 2.16521691e+11 Hz>,
  'flux': <Projection [[0.4863091 , 0.85207975, 0.4318698 , ..., 1.9904716 ,
                       nan,        nan],
               [0.49557889,        nan,        nan, ...,        nan,
                       nan,        nan],
               [2.3502598 ,        nan,        nan, ...,        nan,
                2.1291647 ,        nan],
               ...,
               [       nan,        nan,        nan, ...,        nan,
                       nan, 0.39898998],
               [       nan,        nan, 0.9190112 , ...,        nan,
                       nan,        nan],
               [       nan,        nan, 1.0553234 , ...,        nan,
                       nan,        nan]] K km / s>,
  'stddev': <Quantity 0.5901029 K>,
  'beam': <Quantity 2.21572103e-12 sr>,
  'euk': <Quantity 106.30154 K>,
  'eujs': <Quantity 1.46765115e-21 J>,
  'degen': 17.0,
  'aij': <Quantity 3.64938767e-05 Hz>,
  'filename': '/blue/adamginsburg/d.jef

In [28]:
compute_cdms_aij(testlgint,testnu,testg,testelo_J,testeup_J,qrot300,300*u.K)

<Quantity [3.72857027e+22] MHz3 nm2 s2 / (J m3)>

In [32]:
mu2S=compute_mu2S(testint,testnu,testg,testelo_J,testeup_J,qrot300,300*u.K)

In [37]:
(mu2S*((64*(np.pi**4)*(testnu**3))/(3*h*(c**3)))).decompose()

<Quantity [3.72856706e+22] s / (kg m3)>

In [40]:
(u.Debye**2).decompose()

Unit("1.11111e-59 A2 m2 s2")

In [42]:
((64*(np.pi**4)*(testnu**3)*1e-5*u.Hz)/(3*h*(c**3))).decompose()

<Quantity 1.18152878e+40 1 / (kg m5)>